# Basic/B2a Example
It is equivalent to the B2a example in Geant4 but re-written with a new more Julia friendly interface. The core of the example is to create an instance of the type `G4JLApplication` with all the essential elements defined (detector and geometry, physics list, primary particles generator, actions, simulated data being accumulated, etc.) 

#### Import the `Geant4` and `Geant4.SystemOfUnits` modules

In [1]:
using Geant4
using Geant4.SystemOfUnits

#### Define the Detector Parameters and Geometry
We include a file to define de structure `B2aDetector` and the construction method `B2aConstruct` to make the notebook more manageable

In [2]:
include(joinpath(@__DIR__, "DetectorB2a.jl"))

#### Define the rest of the application
This includes:
- Simulation data structure `B2SimData` that will be filled with the actions
- Initialize particle gun
- Define stepping and tracking actions in this example that do storage the step positions

In [3]:
#---Define Simulation Data struct------------------------------------------------------------------
using GeometryBasics
mutable struct B2aSimData <: G4JLSimulationData
  tracks::Vector{Vector{Point3{Float64}}}   # vector of vector of step positions
  B2aSimData() = new([])
end

#---Particle Gun initialization--------------------------------------------------------------------
function gun_initialize(gen::G4JLParticleGun, det::G4JLDetector)
  pg = GetGun(gen)
  SetParticleByName(pg, "proton")
  SetParticleEnergy(pg, 3GeV)
  SetParticleMomentumDirection(pg, G4ThreeVector(0,0,1))
  SetParticlePosition(pg, G4ThreeVector(0,0,-16cm))
  SetParticlePosition(pg, G4ThreeVector(0, 0, -det.worldZHalfLength))
end
Geant4.getInitializer(::G4JLParticleGun) = gun_initialize

#---Step action------------------------------------------------------------------------------------
function stepaction(step::G4Step, app::G4JLApplication)::Nothing
  tracks = app.simdata.tracks
  p = GetPosition(GetPostStepPoint(step))
  push!(tracks[end], Point3{Float64}(x(p),y(p),z(p)))
  nothing
end
#---Tracking pre-action---------------------------------------------------------------------------- 
function pretrackaction(track::G4Track, app::G4JLApplication)::Nothing
  tracks = app.simdata.tracks
  p = GetPosition(track)[]
  push!(tracks, [Point3{Float64}(x(p),y(p),z(p))])
  nothing
end
#---Tracking post-action---------------------------------------------------------------------------
function posttrackaction(track::G4Track, ::G4JLApplication)::Nothing
  println("Finished track ID $(GetTrackID(track))")
  nothing
end

posttrackaction (generic function with 1 method)

#### Instantiate a `G4JLApplication` with all the elements
The G4RunManager is created at this moment

In [4]:
#---Create the Application-------------------------------------------------------------------------
app = G4JLApplication(B2aDetector(nChambers=5);                   # detector with parameters
                      simdata = B2aSimData(),                     # simulation data structure
                      runmanager_type = G4RunManager,             # what RunManager to instantiate
                      physics_type = FTFP_BERT,                   # what physics list to instantiate
                      generator_type = G4JLParticleGun,           # what primary generator to instantiate
                      stepaction_method = stepaction,             # step action method
                      pretrackaction_method = pretrackaction,     # pre-tracking action
                      posttrackaction_method = posttrackaction)   # post-tracking action              


**************************************************************
 Geant4 version Name: geant4-11-01-patch-01 [MT]   (10-February-2023)
                       Copyright : Geant4 Collaboration
                      References : NIM A 506 (2003), 250-303
                                 : IEEE-TNS 53 (2006), 270-278
                                 : NIM A 835 (2016), 186-225
                             WWW : http://geant4.org/
**************************************************************



G4JLApplication(Geant4.G4RunManagerAllocated(Ptr{Nothing} @0x00007fe59b0de600), B2aDetector(5, true, 800.0, 200.0, 50.0, 2940.0), B2aSimData(Vector{Point3{Float64}}[]), G4RunManager, G4JLDetectorConstruction, FTFP_BERT, G4JLParticleGun, G4JLTrackingAction, G4JLSteppingAction, stepaction, pretrackaction, posttrackaction)

#### Finally,  configure the application, initialize the run and fire the beam for 1 event

In [5]:
configure(app)

<<< Geant4 Physics List simulation engine: FTFP_BERT



In [6]:
initialize(app)

Checking overlaps for volume Target:0 (G4Tubs) ... OK! 
Checking overlaps for volume Tracker:0 (G4Tubs) ... OK! 
Checking overlaps for volume Chamber_PV:1 (G4Tubs) ... OK! 
Checking overlaps for volume Chamber_PV:2 (G4Tubs) ... OK! 
Checking overlaps for volume Chamber_PV:3 (G4Tubs) ... OK! 
Checking overlaps for volume Chamber_PV:4 (G4Tubs) ... OK! 
Checking overlaps for volume Chamber_PV:5 (G4Tubs) ... OK! 

 hInelastic FTFP_BERT : threshold between BERT and FTFP is over the interval 
 for pions :   3 to 6 GeV
 for kaons :   3 to 6 GeV
 for proton :  3 to 6 GeV
 for neutron : 3 to 6 GeV

### Adding tracking cuts for neutron  TimeCut(ns)= 10000  KinEnergyCut(MeV)= 0


In [14]:
beamOn(app, 1)

Finished track ID 1
Finished track ID 88
Finished track ID 87
Finished track ID 86
Finished track ID 85
Finished track ID 84
Finished track ID 83
Finished track ID 82
Finished track ID 81
Finished track ID 80
Finished track ID 89
Finished track ID 79
Finished track ID 78
Finished track ID 77
Finished track ID 95
Finished track ID 94
Finished track ID 93
Finished track ID 92
Finished track ID 91
Finished track ID 90
Finished track ID 76
Finished track ID 75
Finished track ID 74
Finished track ID 73
Finished track ID 72
Finished track ID 71
Finished track ID 70
Finished track ID 69
Finished track ID 68
Finished track ID 67
Finished track ID 66
Finished track ID 108
Finished track ID 107
Finished track ID 106
Finished track ID 105
Finished track ID 104
Finished track ID 103
Finished track ID 102
Finished track ID 101
Finished track ID 100
Finished track ID 99
Finished track ID 98
Finished track ID 97
Finished track ID 96
Finished track ID 65
Finished track ID 64
Finished track ID 109
Fini

In [12]:
using LinearAlgebra
tracks = app.simdata.tracks
lenghts = [ norm(t[end]-t[begin]) for t in tracks]

120-element Vector{Float64}:
 5880.180983049687
    0.12571254311932245
    0.025057132944776263
    1.2201371924612012
    0.07427318633565101
    0.05928577327799816
    0.03160502010038563
    0.028775544916542946
    0.028944462550383063
    0.02565660995434016
    ⋮
    0.09854067748622153
    0.23972400324081605
    0.3009055520337624
  630.7404198213954
    0.08385862177138537
    0.028139109111267428
    0.023582705188809252
    6.34418533156728
    0.2713584258470664